In [17]:
import pandas as pd

data = pd.read_csv('D:/research/data/IMDB_Dataset.csv', error_bad_lines=False)
data.sample(n=10)

C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_25224\1129593017.py:3: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  data = pd.read_csv('D:/research/data/IMDB_Dataset.csv', error_bad_lines=False)


,review,sentiment
13441,"I do not recommend this movie , because it's i...",negative
2016,I've been trying to find out about this series...,positive
3535,"This satire is just really, really dead-on, an...",positive
39364,When I was 13 or so I was lucky enough to find...,negative
38714,Everyone who has ever wondered how to make a f...,positive
10091,I found the episodes to be fascinating and wel...,positive
48688,Rarely does a film capture such intense drama ...,positive
35092,"with that, carry the same dark weaknesses we a...",positive
27211,This movie is probably one of 3 worst movies m...,negative
40915,A party-hardy frat boy's sister is brutally mu...,negative


In [18]:
data['label'] = data['sentiment'].replace(['positive', 'negative'],['1', '0'])

In [19]:
data.sample(n=10)

,review,sentiment,label
36146,"Sorry, but I usually love French thrillers - e...",negative,0
23919,"Dreamgirls, despite its fistful of Tony wins i...",negative,0
32592,Raising Victor Vargas: A Review<br /><br />You...,negative,0
35994,This is the ultimate Kung Fu movie! This is th...,negative,0
47242,"""Come Undone"" appears to elicit a lot of opini...",positive,1
48261,I'm surprised by some of the comments on this ...,positive,1
22094,if you're a sucker for corny movies and are lo...,negative,0
25731,"Okay, here's what I think about Jack Frost. I ...",positive,1
5841,This is the movie that epitomizes the D&D fear...,negative,0
46145,Successful films on metaphysical subjects are ...,positive,1


In [24]:
data['label'].value_counts()

1    25000
0    25000
Name: label, dtype: int64

In [23]:
data.to_csv('D:/research/data/IMDB_Dataset_label.csv')

In [28]:
#Text Preprocessing for sentiment analysis
import string
import emoji
import re
import nltk
from textblob import TextBlob
from nltk.corpus import wordnet
from nltk.metrics import edit_distance
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize
import pandas as pd
stopwords = nltk.corpus.stopwords.words('english')

def preprocess(text):
    
    #1. Generating the list of words in the tweet (hastags and other punctuations removed)
    text_blob = TextBlob(text)
    text = ' '.join(text_blob.words)
    
    #2. clean the number 
    text = re.sub(r'[0-9]', '', text)
    
    #3. lower the text
    text = text.lower()
    
    #4. conver the emoji to text form
    text = emoji.demojize(text)
    
    #5. remove punctuation 
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '')
    
    #6. tokenize the text
    text = word_tokenize(text)
    
    #7. remove empty token
    text = [t for t in text if len(t) > 0]
    
    #8. remove non-alphabetical token
    text = [t for t in text if t.isalpha()]
    
    #9. replace the negation token
    replacer  = AntonymReplacer()
    text = replacer.replace_negations(text)
    
    #10. remove the stopwords
    text = [i for i in text if i not in stopwords]
    
    #11. stem the text
    porter_stemmer = PorterStemmer()
    text = [porter_stemmer.stem(w) for w in text]
    
    return text

class AntonymReplacer(object):
    def replace(self, word, pos=None):
        antonyms = set()

        for syn in wordnet.synsets(word, pos=pos):
            for lemma in syn.lemmas():
                for antonym in lemma.antonyms():
                    antonyms.add(antonym.name())

        if len(antonyms) == 1:
            return antonyms.pop()
        else:
            return None

    def replace_negations(self, sent):
        i, l = 0, len(sent)
        words = []

        while i < l:
            word = sent[i]

            if word == 'not' and i+1 < l:
                ant = self.replace(sent[i+1])

                if ant:
                    words.append(ant)
                    i += 2
                    continue

            words.append(word)
            i += 1

        return words

In [30]:
print(data['review'].iloc[12])
print(preprocess(data['review'].iloc[12]))

So im not a big fan of Boll's work but then again not many are. I enjoyed his movie Postal (maybe im the only one). Boll apparently bought the rights to use Far Cry long ago even before the game itself was even finsished. <br /><br />People who have enjoyed killing mercs and infiltrating secret research labs located on a tropical island should be warned, that this is not Far Cry... This is something Mr Boll have schemed together along with his legion of schmucks.. Feeling loneley on the set Mr Boll invites three of his countrymen to play with. These players go by the names of Til Schweiger, Udo Kier and Ralf Moeller.<br /><br />Three names that actually have made them selfs pretty big in the movie biz. So the tale goes like this, Jack Carver played by Til Schweiger (yes Carver is German all hail the bratwurst eating dudes!!) However I find that Tils acting in this movie is pretty badass.. People have complained about how he's not really staying true to the whole Carver agenda but we on

In [31]:
from sklearn.feature_extraction.text import TfidfVectorizer

"""
min_df=2, discard words appearing in less than 2 documents
max_df=0.9, discard words appering in more than 90% of the documents
sublinear_tf=True, use sublinear weighting
use_idf=True, enable IDF
"""
vec = TfidfVectorizer(
    analyzer=preprocess,
    min_df=2,
    max_df=0.9,
    sublinear_tf=True,
    use_idf=True
)

tfidf_model = vec.fit(data['review'])
train_vec = vec.transform(data['review'])

In [32]:
import pickle

#save vectorizer
vectorizer_file = 'D:/research/models/tfidf_vectorizer.sav';
pickle.dump(tfidf_model, open(vectorizer_file, 'wb'))

In [33]:
from sklearn.model_selection import train_test_split
SEED = 4000

X_train, X_test, y_train, y_test = train_test_split(train_vec, 
                                            data.label, test_size=0.2, random_state=SEED)

In [34]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

#X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.33, random_state=39)

print('X_train shape: {}'.format(X_train.shape))
print('X_test shape: {}'.format(X_test.shape))
print('y_train shape: {}'.format(y_train.shape))
print('y_test shape: {}'.format(y_test.shape))

# pick Multinomial Naive Bayes and RandomForest algorithms 
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier

clf_mnb = MultinomialNB()
clf_rfc = RandomForestClassifier(random_state=39)

clf_names = ['MultinomialNB', 'RandomForest']
clf_types = [clf_mnb, clf_rfc]

for (i, clf) in enumerate(clf_types):
    clf.fit(X_train, y_train.values.ravel())
    print('Result of {}\n'.format(clf_names[i]))
    predictions = clf.predict(X_test)
    print(classification_report(predictions, y_test))
    print('\n')
    print('Confusion matrix: \n', confusion_matrix(predictions, y_test))
    print('\n')
    print('Accuracy score: ', accuracy_score(predictions, y_test))
    print('\n\n\n')

X_train shape: (40000, 45394)
X_test shape: (10000, 45394)
y_train shape: (40000,)
y_test shape: (10000,)
Result of MultinomialNB

              precision    recall  f1-score   support

           0       0.87      0.85      0.86      5085
           1       0.85      0.87      0.86      4915

    accuracy                           0.86     10000
   macro avg       0.86      0.86      0.86     10000
weighted avg       0.86      0.86      0.86     10000



Confusion matrix: 
 [[4344  741]
 [ 662 4253]]


Accuracy score:  0.8597




Result of RandomForest

              precision    recall  f1-score   support

           0       0.85      0.86      0.85      5001
           1       0.85      0.85      0.85      4999

    accuracy                           0.85     10000
   macro avg       0.85      0.85      0.85     10000
weighted avg       0.85      0.85      0.85     10000



Confusion matrix: 
 [[4276  725]
 [ 730 4269]]


Accuracy score:  0.8545






In [37]:
import pickle

# save the model to disk
filename1 = 'D:/research/models/MultinomialNB_sentiment_model.sav'
pickle.dump(clf_mnb, open(filename1, 'wb'))

filename2 = 'D:/research/models/RandomForest_sentiment_model.sav'
pickle.dump(clf_rfc, open(filename2, 'wb'))

In [38]:
import pickle

filename1 = 'D:/research/models/MultinomialNB_sentiment_model.sav' #load trained model
loaded_clf_mnb_model = pickle.load(open(filename1, 'rb'))

filename2 = 'D:/research/models/RandomForest_sentiment_model.sav'#load trained model
loaded_clf_rfc_model = pickle.load(open(filename2, 'rb'))

tfidf_vectorizer = 'D:/research/models/tfidf_vectorizer.sav';
loaded_tfidf_vectorizer = pickle.load(open(tfidf_vectorizer, 'rb'))#load save vectorizer

In [39]:
tweet = 'We are providing better care, and more choice, at lower cost.'
preprocess_tweet = preprocess(tweet)
print(preprocess_tweet)

['provid', 'better', 'care', 'choic', 'lower', 'cost']


In [40]:
# vectorize the tweet
tweet_vec = loaded_tfidf_vectorizer.transform(pd.Series([tweet]))

# predict a label using Multinomial Naive Bayes
tweet_prediction = loaded_clf_mnb_model.predict(tweet_vec.toarray())
tweet_prediction = 'positive' if tweet_prediction[0] == '1' else 'negative'
print('"{}" has been predicted for the tweet: {}'.format(tweet_prediction, tweet))

"negative" has been predicted for the tweet: We are providing better care, and more choice, at lower cost.


In [41]:
# predict a label using RandomForest

tweet_prediction = loaded_clf_rfc_model.predict(tweet_vec.toarray())
tweet_prediction = 'positive' if tweet_prediction[0] == '1' else 'negative'
print('"{}" has been predicted for the tweet: {}'.format(tweet_prediction, tweet))

"positive" has been predicted for the tweet: We are providing better care, and more choice, at lower cost.
